Install Required Libraries

In [1]:
# pip install lightgbm

In [2]:
# pip install pandas-gbq

In [3]:
# pip install google-cloud-bigquery

Import Required Libraries

In [4]:
from google.cloud import bigquery
import pandas as pd
import pandas_gbq
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import lightgbm as lgb

/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


Set Environment variable for Google Credentials

In [5]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getcwd() + "/Key.json"

Create Big Query Client

In [6]:
bg_client = bigquery.Client(project='bigquery-public-data')

Get the data set and display all the tables from the given dataset

In [7]:
data_set_ref = bg_client.dataset('chicago_taxi_trips', project='bigquery-public-data')
data_set = bg_client.get_dataset(data_set_ref)
for tab in bg_client.list_tables(data_set):
    print(tab.table_id)

taxi_trips


List all the columns and the corresponding details for the given table

In [8]:
tab = bg_client.get_table(data_set.table('taxi_trips'))
tab.schema

[SchemaField('unique_key', 'STRING', 'REQUIRED', 'Unique identifier for the trip.', (), None),
 SchemaField('taxi_id', 'STRING', 'REQUIRED', 'A unique identifier for the taxi.', (), None),
 SchemaField('trip_start_timestamp', 'TIMESTAMP', 'NULLABLE', 'When the trip started, rounded to the nearest 15 minutes.', (), None),
 SchemaField('trip_end_timestamp', 'TIMESTAMP', 'NULLABLE', 'When the trip ended, rounded to the nearest 15 minutes.', (), None),
 SchemaField('trip_seconds', 'INTEGER', 'NULLABLE', 'Time of the trip in seconds.', (), None),
 SchemaField('trip_miles', 'FLOAT', 'NULLABLE', 'Distance of the trip in miles.', (), None),
 SchemaField('pickup_census_tract', 'INTEGER', 'NULLABLE', 'The Census Tract where the trip began. For privacy, this Census Tract is not shown for some trips.', (), None),
 SchemaField('dropoff_census_tract', 'INTEGER', 'NULLABLE', 'The Census Tract where the trip ended. For privacy, this Census Tract is not shown for some trips.', (), None),
 SchemaField('

In [9]:
tab.num_rows

194069509

Create a Sample query and assign the data into Pandas Dataframe

In [10]:
QUERY = """
    SELECT EXTRACT(YEAR FROM trip_start_timestamp) AS YEAR, COUNT(unique_key) as TRIPS
    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
    GROUP BY YEAR
        """
df=pd.read_gbq(QUERY)
df

Downloading: 100%|██████████| 8/8 [00:00<00:00, 36.45rows/s]


,YEAR,TRIPS
0,2019,16477365
1,2015,32385875
2,2017,24988003
3,2013,27217716
4,2014,37395436
5,2016,31759339
6,2018,20732088
7,2020,3113687


# Get Data

Lets take the data corresponding to Feb-2020 to build a base model

In [11]:
QUERY = """
    SELECT *
    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
    WHERE EXTRACT(YEAR FROM trip_start_timestamp) = 2020
    AND EXTRACT(MONTH FROM trip_start_timestamp) = 2
    """

df=pd.read_gbq(QUERY)
df.head(5)

Downloading: 100%|██████████| 1122124/1122124 [04:58<00:00, 3758.04rows/s]


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,ba8e167fd614d76bec89951efc3c2439e321e767,f1ed8794238deb1c8f7c3759aa096ae20e3e769cc4c9ad...,2020-02-01 00:00:00+00:00,2020-02-01 00:15:00+00:00,593.0,2.80,NaN,NaN,76.0,NaN,...,4.0,17.70,Credit Card,Metro Jet Taxi A,41.980264,-87.913625,POINT (-87.913624596 41.9802643146),NaN,NaN,None
1,af1e6f3a4b3b0a2d981fd6493d80fcc3fd6f0ffd,51d13e9ec1a04be36d87a3746e0e4f8c68ca5ed59e6f7a...,2020-02-01 00:00:00+00:00,2020-02-01 00:00:00+00:00,710.0,1.80,NaN,NaN,8.0,8.0,...,1.0,9.75,Cash,City Service,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.899602,-87.633308,POINT (-87.6333080367 41.899602111)
2,c085286c289f4d2d20b6991bff839875aabb74bd,d4578429775c32e03fb5139e1d132b91be7e27af9675e7...,2020-02-01 00:00:00+00:00,2020-02-01 00:15:00+00:00,600.0,1.40,NaN,NaN,28.0,8.0,...,0.0,10.00,Credit Card,Taxi Affiliation Services,41.874005,-87.663518,POINT (-87.6635175498 41.874005383),41.899602,-87.633308,POINT (-87.6333080367 41.899602111)
3,70f792072de11a5aa98f12b73e5abc9c2e235474,1147903107a545f4f4bdd87989e636c520c7f9a08f8342...,2020-02-01 00:00:00+00:00,2020-02-01 00:30:00+00:00,1704.0,16.94,NaN,NaN,76.0,8.0,...,5.0,58.20,Credit Card,Sun Taxi,41.980264,-87.913625,POINT (-87.913624596 41.9802643146),41.899602,-87.633308,POINT (-87.6333080367 41.899602111)
4,8ff7e9098cdff720d905df0c7e285c1287758b1b,200712b5d1ed69a162d0cd453bb41d2d9ef515c5c6995d...,2020-02-01 00:00:00+00:00,2020-02-01 00:30:00+00:00,1685.0,18.73,NaN,NaN,76.0,8.0,...,4.0,61.20,Credit Card,Chicago Carriage Cab Corp,41.980264,-87.913625,POINT (-87.913624596 41.9802643146),41.899602,-87.633308,POINT (-87.6333080367 41.899602111)


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122124 entries, 0 to 1122123
Data columns (total 23 columns):
 #   Column                  Non-Null Count    Dtype              
---  ------                  --------------    -----              
 0   unique_key              1122124 non-null  object             
 1   taxi_id                 1122124 non-null  object             
 2   trip_start_timestamp    1122124 non-null  datetime64[ns, UTC]
 3   trip_end_timestamp      1122076 non-null  datetime64[ns, UTC]
 4   trip_seconds            1121943 non-null  float64            
 5   trip_miles              1122118 non-null  float64            
 6   pickup_census_tract     711601 non-null   float64            
 7   dropoff_census_tract    709129 non-null   float64            
 8   pickup_community_area   1042701 non-null  float64            
 9   dropoff_community_area  1021866 non-null  float64            
 10  fare                    1122014 non-null  float64            
 11  tips       

For Model purpose, lets choose only the below fields
1. trip_start_timestamp
2. pickup_latitude, pickup_longitude
3. dropoff_latitude, dropoff_longitude
4. compare
5. fare - This field will be our label to predict

In [13]:
df_for_model = df[['trip_start_timestamp','pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude','company','fare']]
df_for_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122124 entries, 0 to 1122123
Data columns (total 7 columns):
 #   Column                Non-Null Count    Dtype              
---  ------                --------------    -----              
 0   trip_start_timestamp  1122124 non-null  datetime64[ns, UTC]
 1   pickup_latitude       1042806 non-null  float64            
 2   pickup_longitude      1042806 non-null  float64            
 3   dropoff_latitude      1024316 non-null  float64            
 4   dropoff_longitude     1024316 non-null  float64            
 5   company               1122124 non-null  object             
 6   fare                  1122014 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(5), object(1)
memory usage: 59.9+ MB


In [14]:
df_for_model=df_for_model.dropna()
df_for_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1010127 entries, 1 to 1122123
Data columns (total 7 columns):
 #   Column                Non-Null Count    Dtype              
---  ------                --------------    -----              
 0   trip_start_timestamp  1010127 non-null  datetime64[ns, UTC]
 1   pickup_latitude       1010127 non-null  float64            
 2   pickup_longitude      1010127 non-null  float64            
 3   dropoff_latitude      1010127 non-null  float64            
 4   dropoff_longitude     1010127 non-null  float64            
 5   company               1010127 non-null  object             
 6   fare                  1010127 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(5), object(1)
memory usage: 61.7+ MB


Set Jupyter notbook to display all fields

In [15]:
pd.options.display.max_columns = None

Find if we have any records that have fare value of less than minimum taxi fare in Chicago, which is $2.70 and remove those rows as well 

In [16]:
df_for_model = df_for_model.loc[df_for_model['fare'] >= 2.70]
df_for_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1009336 entries, 1 to 1122123
Data columns (total 7 columns):
 #   Column                Non-Null Count    Dtype              
---  ------                --------------    -----              
 0   trip_start_timestamp  1009336 non-null  datetime64[ns, UTC]
 1   pickup_latitude       1009336 non-null  float64            
 2   pickup_longitude      1009336 non-null  float64            
 3   dropoff_latitude      1009336 non-null  float64            
 4   dropoff_longitude     1009336 non-null  float64            
 5   company               1009336 non-null  object             
 6   fare                  1009336 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(5), object(1)
memory usage: 61.6+ MB


Add new columns to indicate day of the week and hour of the day columns

In [17]:
df_for_model['trip_day_of_week']=df_for_model['trip_start_timestamp'].dt.dayofweek
df_for_model['trip_hour_of_day']=df_for_model['trip_start_timestamp'].dt.hour
df_for_model=df_for_model.drop(['trip_start_timestamp'],axis=1)
df_for_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1009336 entries, 1 to 1122123
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   pickup_latitude    1009336 non-null  float64
 1   pickup_longitude   1009336 non-null  float64
 2   dropoff_latitude   1009336 non-null  float64
 3   dropoff_longitude  1009336 non-null  float64
 4   company            1009336 non-null  object 
 5   fare               1009336 non-null  float64
 6   trip_day_of_week   1009336 non-null  int64  
 7   trip_hour_of_day   1009336 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 69.3+ MB


Company field is a categorical value, so lets add dummy variables

In [18]:
df_dummy = pd.get_dummies(df_for_model['company'],prefix='comp', drop_first=True)
df_for_model = pd.concat([df_for_model,df_dummy],axis=1)
df_for_model=df_for_model.drop('company',axis=1)
df_for_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1009336 entries, 1 to 1122123
Data columns (total 51 columns):
 #   Column                                             Non-Null Count    Dtype  
---  ------                                             --------------    -----  
 0   pickup_latitude                                    1009336 non-null  float64
 1   pickup_longitude                                   1009336 non-null  float64
 2   dropoff_latitude                                   1009336 non-null  float64
 3   dropoff_longitude                                  1009336 non-null  float64
 4   fare                                               1009336 non-null  float64
 5   trip_day_of_week                                   1009336 non-null  int64  
 6   trip_hour_of_day                                   1009336 non-null  int64  
 7   comp_24 Seven Taxi                                 1009336 non-null  uint8  
 8   comp_2733 - 74600 Benny Jona                       1009336 non

Split the data into training and test set

In [19]:
X=df_for_model.drop(['fare'],axis=1)
y=df_for_model['fare']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20,random_state=123)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(807468, 50)
(201868, 50)
(807468,)
(201868,)


For this project, lets try the below models and get the Root Mean Squared error (RMSE) for each model
1. LightGBM
2. Random Forest
3. Linear Regression
4. Gradient Boosting

Since the dataset is huge we dont have to do K-Fold

In [21]:
lgbm_Data = lgb.Dataset(X_train, y_train)
params = {}
params['learning_rate'] = 0.003
params['objective'] = 'regression'
params['metric'] = 'rmse'
num_round=50
lgbm_model = lgb.train(params,lgbm_Data, num_round)
lgbr_predict=lgbm_model.predict(X_test)
lgbr_srsme = np.sqrt(mean_squared_error(lgbr_predict, y_test))
print('RMSE for Light GBM is ', lgbr_srsme)

RMSE for Light GBM is  99.03031301773454


In [23]:
rfr = RandomForestRegressor(n_estimators = 50, n_jobs=-1)
rfr.fit(X_train, y_train)
rfr_predict = rfr.predict(X_test)
rfr_srsme = np.sqrt(mean_squared_error(rfr_predict, y_test))
print('RMSE for Random Forest is ', rfr_srsme)

RMSE for Random Forest is  104.99145073297777


In [24]:
LR = LinearRegression()
LR.fit(X_train, y_train)
LR_predict = LR.predict(X_test)
LR_srsme = np.sqrt(mean_squared_error(LR_predict, y_test))
print('RMSE for Linear Regression is ', LR_srsme)

RMSE for Linear Regression is  96.55851420753619


In [25]:
GBR = GradientBoostingRegressor(n_estimators=50)
GBR.fit(X_train, y_train)
GBR_predict = LR.predict(X_test)
GBR_srsme = np.sqrt(mean_squared_error(GBR_predict, y_test))
print('RMSE for Gradient Boosting is ', GBR_srsme)

RMSE for Gradient Boosting is  96.55851420753619


Lets Calculate the baseline prediction and cross verify it against what we got

In [26]:
avg_fare=round(np.mean(y_train),2)
baseline_pred=np.repeat(avg_fare,y_test.shape[0])
baseline_rmse=np.sqrt(mean_squared_error(baseline_pred, y_test))
print("Basline RMSE of Validation data :",baseline_rmse)

Basline RMSE of Validation data : 100.89574187592116


Based on the above baseline, it seems LGBM is giving the best prediction.  Lets take that as our prototype